In [1]:
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from dotenv import load_dotenv, find_dotenv
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple
import numpy as np

Load the VectorDatabase

In [2]:
load_dotenv(find_dotenv())
embeddings = OpenAIEmbeddings()
db = Chroma(persist_directory='./chroma_db', embedding_function=embeddings)

Initialize the LLM and RAG Retrieval

In [3]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.8, max_tokens=3500)
retriever = db.as_retriever(k=7)
rag = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=db.as_retriever())

#### Building The User's Preference Interface

We will prepare a buyers match preferences and human input for some AI questions, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language.

In [17]:
ai_questions = ["What's your budget range for a new home?",
             "Are you looking for a newly built home or an older property with character?", 
             "Do you have any specific architectural style preferences?", 
             "How important is it for you to have outdoor recreational facilities nearby?",
             "How many rooms do you want in your home?",
             "Are you considering the resale value of the property in your decision-making process?"
            ]
human_answers = [
    "My budget range is between $80,000 to $650,000.",
    "I'm open to both options, but I lean towards a newly built home for its modern amenities.",
    "I'm particularly interested in contemporary architecture with clean lines and plenty of natural light.",
    "Outdoor recreational facilities like parks or trails are crucial for me as I enjoy outdoor activities like hiking and biking.",
    "I prefer between 2 or 3 and also 2 bathrooms.",
    "Yes, resale value is important to me. I'm looking for a property in an area with potential for appreciation over time."
]

Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database

Let's setup a chat history between you and AI where we provide your answers to the questions AI "asked"

In [18]:
n = len(ai_questions)
history = ChatMessageHistory()
history.add_user_message(f"""You are AI that will recommend user a home based on their answers to personal questions. Ask user {n} questions""")

# Add questions and answers to the history
[(history.add_ai_message(ai_questions[i]), history.add_user_message(human_answers[i])) for i in range(n)]
history.add_ai_message("""Now tell me a summary of homes you're considering to buy, and specify how you want me to respond to you with the \
price, sqm, rooms, bathrooms, parkings, neighborhood and a summary of the neighborhood description.""")

Now, we want get the recommendations of homes, pass them to LLM and see how it would the house q/a inputs in all movie plots and the recommended results.

Eventually we will trigger an max_token limit so we developed a memory summary conversation that would hold a summary of our conversation and AI recommendations

In [19]:
# TODO: create a memory that will have a summary of the recommendations
summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key='home_summary',
    input_key='input',
    buffer=f"The human answered {n} personal questions. Use them to list the best places, rate them from 1 to 10, and include the price and location \
    comparing how much he probably will like it to live.",
    return_messages=True
)

# you could choose to store some of the q/a in memory as well, in addition to original questions 
# we're storing AI messages in memory - they'll help LLM to generate a better final recommendation
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)
        
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key='question_and_answers',
    input_key='input'
)

In [20]:
for m in conversational_memory.chat_memory.messages:
    print(m.content)

You are AI that will recommend user a home based on their answers to personal questions. Ask user 6 questions
What's your budget range for a new home?
My budget range is between $80,000 to $650,000.
Are you looking for a newly built home or an older property with character?
I'm open to both options, but I lean towards a newly built home for its modern amenities.
Do you have any specific architectural style preferences?
I'm particularly interested in contemporary architecture with clean lines and plenty of natural light.
How important is it for you to have outdoor recreational facilities nearby?
Outdoor recreational facilities like parks or trails are crucial for me as I enjoy outdoor activities like hiking and biking.
How many rooms do you want in your home?
I prefer between 2 or 3 and also 2 bathrooms.
Are you considering the resale value of the property in your decision-making process?
Yes, resale value is important to me. I'm looking for a property in an area with potential for appr

In [21]:
memory = CombinedMemory(memories=[conversational_memory, summary_memory])

Let's create a PromptTemplate that would hold continuously updating summary of our conversation, our personal Q/A, and a placeholder for movie plot for AI to rate. Think about how you can pass your questions and answers into the template - there are many different ways to do it.

In [22]:
HOME_MATCH_TEMPLATE = """The following is an informative conversation between a human and an AI Home Match Lister and Recommender. 
                        The AI is follows human instructions and provides movie ratings for a human based on the movie plot
                        and human's persona derived from their answers to questions. 

Summary of Recommendations:
{home_summary}
Personal Questions and Answers:
{question_and_answers}
Human: {input}
AI:"""
prompt = PromptTemplate(
    input_variables=["home_summary", "input", "question_and_answers"],
    template=HOME_MATCH_TEMPLATE
)

In [23]:
# create a recommendation conversation chain that will let us ask AI for recommendations on all movies
recommender = ConversationChain(llm=llm, verbose=True, memory=memory, prompt=prompt)

Pick a list of places you would like to live and the LLM doesn't know how to respond

In [24]:
# list of recent movies that you'd like AI to consider when recommending a movie
human_neighborhoods = [ "Parque Lefevre", "Punta Paitilla", "Punta Pacifica", "Bella Vista"] 

Query the vector database for the most 5 semantically similar chunks

In [25]:
neighborhood = np.random.choice(human_neighborhoods)
query = f"""List apartments or homes and all the info related from neighborhood of {neighborhood}, include all the description and the appartment image.
    """
result = rag.invoke(query) # find top 5 semantically similar documents to the query
print(result['query'])
print()
print(result['result'])

List apartments or homes and all the info related from neighborhood of Punta Pacifica, include all the description and the appartment image.
    

I found information about an apartment in Punta Pacifica, Panama, with the following details:

- **Apartment Description**: Alquiler de apartamento, Discover luxury at its finest in this fully furnished apartment in Punta Pacifica. With its charming balcony, 2 exquisitely designed rooms, 2 modern bathrooms, plus a comfortable maid's room and bathroom. The open kitchen adds a touch of elegance. Live the style and comfort you deserve! Ver datos
- **Square Meters (sqm)**: 109
- **Number of Rooms**: 2
- **Number of Bathrooms**: 2
- **Parking**: 1
- **Neighborhood**: Punta Pacifica
- **Price**: $235,000
- **Neighborhood Description**: Punta Pacifica in Panama is an exclusive and luxurious neighborhood that offers a unique combination of comfort and luxury. With its stunning sea views and modern skyscrapers, it is the ideal place for those seeking

In [26]:
for neighborhood in human_neighborhoods:
    print("Neighborhood: " + neighborhood)
    query = f"""List 5 apartments or homes and all the info related from neighborhood of {neighborhood}, 
    include all the information and most importantly the appartment image and contact information.
    """
    result = rag.invoke(query) # find top 5 semantically similar documents to the query
    home_results = result['result']
    summary_evaluation = f"""
         =====================================
        === START HOME MATCH SUMMARY FOR {neighborhood} ===
        {home_results}
        === END NEIGHBORHOOD HOME MATCH SUMMARY ===
        =====================================
        
        RATING INSTRUCTIONS THAT MUST BE STRICTLY FOLLOWED:
        AI will provide a highly personalized evaluation of homes or apartments based only on the home match summary human provided 
        and human answers to questions included with the context. 
        AI should be very sensible to human personal preferences captured in the answers to personal questions, 
        and should not be influenced by anything else.
        AI will also build a persona for human based on human answers to questions, and use this persona to recommend and list possible home or
        apartments that matches the preferences of the human.
        OUTPUT FORMAT:
        First, include that persona you came up with in the explanation for the home or apartment evaluation. Describe the persona in a few sentences.
        Explain how human preferences captured in the answers to personal questions influenced creation of this persona.
        In addition, consider other things like price, neighborhood description, location, contact information, security for this human that you might have as \
        they might give you more information about human's preferences also include the appartmet image.
        Your goal is to provide the best recommendations that are as close as possible to the prices and neighborhoods the human would preffer to live or buy.
        Remember that human has a strong beleif in budget and wants a peacefull place to live, present him something he would probably buy and like it, \
        so your final evaluation should be as precise as possible.
        Listings of apartments and homes will range from the budget presented. 
        You will include a logical explanation for your decis based on human persona you've build and human responses to questions.       
        YOUR LISTING MUST END WITH THE LIST OF THE SELECTED HOME OR APARTMENT AND ALL THE INFORMATION INCLDING THE APARTMENT IMAGE
        IMPORTANT: FOLLOW THE INSTRUCTIONS STRICTLY, OTHERWISE HUMAN WILL NOT BE ABLE TO UNDERSTAND YOUR REVIEW.         
    """
    prediction = recommender.predict(input=summary_evaluation)
    print("=== PREDICTION ===")
    print(prediction)
    print("=== OTHER OPTIONS ARE ===")
    print(home_results)

Neighborhood: Parque Lefevre


> Entering new ConversationChain chain...
Prompt after formatting:
The following is an informative conversation between a human and an AI Home Match Lister and Recommender. 
                        The AI is follows human instructions and provides movie ratings for a human based on the movie plot
                        and human's persona derived from their answers to questions. 

Summary of Recommendations:
[SystemMessage(content='The human answered 6 personal questions. Use them to list the best places, rate them from 1 to 10, and include the price and location     comparing how much he probably will like it to live.')]
Personal Questions and Answers:
Human: You are AI that will recommend user a home based on their answers to personal questions. Ask user 6 questions
AI: What's your budget range for a new home?
Human: My budget range is between $80,000 to $650,000.
AI: Are you looking for a newly built home or an older property with character?
Human: I'